In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('sales_products_abc.csv')

In [3]:
# Скрипт для создания массива данных сгенерировоан в gpt-чате с моими комментариями
# Будет произведен abc-анализ, а именно классификация товаров по степени их важности.
# Для рассчета понадобятся результаты по следующим критериям: количество продаж, выручка(без вычета себеса), прибыль(с вычетом)
df

,date,product,quantity,unit_price,cost_price
0,2024-10-01,SSD,15,271.96,149.06
1,2024-10-01,RAM,18,1032.55,864.41
2,2024-10-01,Projector,18,1123.39,721.53
3,2024-10-01,Processor,6,1041.12,904.38
4,2024-10-01,Drone,19,796.93,668.39
...,...,...,...,...,...
1780,2025-04-01,Mouse,19,1690.87,988.92
1781,2025-04-01,Smartphone,20,1803.47,1163.41
1782,2025-04-01,Headphones,2,1350.88,714.86
1783,2025-04-01,External HDD,13,1868.64,1025.96


In [4]:
#доработка таблицы, добавляем столбец с прибылью с одной единици товара margin
df['margin'] = df['unit_price'] - df['cost_price']

In [5]:
#доработка таблицы, выссчитываем и добавляем столбцы revenue(выручка) и profit(прибыль)
df['revenue'] = df['unit_price'] * df['quantity']
df['profit'] = df['margin'] * df['quantity']

In [6]:
#доработка таблицы, группируем по типу товара и выссчитываем суммарный показатель количества, выручки и прибыли
df_grupped = df.groupby('product', as_index=False)\
.agg(sum_cnt = ('quantity', 'sum'), sum_revenue = ('revenue', 'sum'), sum_profit = ('profit', 'sum'))

In [7]:
#проверяем таблицу на предмет корректного выполнения запроса и получения нужных данных
df_grupped.head()

,product,sum_cnt,sum_revenue,sum_profit
0,Camera,579,603070.76,174972.26
1,Charger,625,708411.50,219664.84
2,Drone,679,660306.95,213092.22
3,E-book Reader,672,716982.17,220571.22
4,External HDD,671,677849.81,209695.23


In [8]:
#дорабатываем таблицу, добавляем столбцы с куммулятивной суммой в процентах
df_grupped['cum_sum_cnt'] = df_grupped.sort_values('sum_cnt', ascending=False)['sum_cnt'].cumsum() / sum(df_grupped['sum_cnt'])
df_grupped['cum_sum_revenue'] = df_grupped.sort_values('sum_revenue', ascending=False)['sum_revenue'].cumsum() / sum(df_grupped['sum_revenue'])
df_grupped['cum_sum_profit'] = df_grupped.sort_values('sum_profit', ascending=False)['sum_profit'].cumsum() / sum(df_grupped['sum_profit'])

In [9]:
#далее необходимо проставить группы в зависимости от вклада в общий результат A – до 80%, B – 80-95%, C – остальное.
df_grupped['abc_cnt'] = np.where(df_grupped['cum_sum_cnt'] < 0.7, 'A', np.where(df_grupped['cum_sum_cnt'] < 0.95, 'B', 'C'))
df_grupped['abc_revenue'] = np.where(df_grupped['cum_sum_revenue'] < 0.7, 'A', np.where(df_grupped['cum_sum_revenue'] < 0.95, 'B', 'C'))
df_grupped['abc_profit'] = np.where(df_grupped['cum_sum_profit'] < 0.7, 'A', np.where(df_grupped['cum_sum_profit'] < 0.95, 'B', 'C'))

In [41]:
#выводим итоговую таблицу с нужными столбцами для подведения итогов abc-анализа
df_abc = df_grupped[['product','abc_cnt', 'abc_revenue', 'abc_profit']].sort_values('abc_profit')
df_abc

,product,abc_cnt,abc_revenue,abc_profit
14,Mouse,A,A,A
21,SSD,A,A,A
22,Smart Light,A,A,A
28,USB Drive,A,A,A
23,Smartphone,A,A,A
25,Speaker,B,A,A
10,Laptop,A,A,A
20,Router,A,B,A
26,TV,A,A,A
7,Graphics Card,A,A,A


In [42]:
#добавляем цветовую заливку ячеек с маркерами анализа для наглядности
df_abc.style.map\
(lambda x: 'background-color: lightgreen' if x == 'A' else 'background-color: pink' if x == 'C' \
 else 'background-color: yellow' ,subset=['abc_cnt', 'abc_revenue', 'abc_profit'])

,product,abc_cnt,abc_revenue,abc_profit
14,Mouse,A,A,A
21,SSD,A,A,A
22,Smart Light,A,A,A
28,USB Drive,A,A,A
23,Smartphone,A,A,A
25,Speaker,B,A,A
10,Laptop,A,A,A
20,Router,A,B,A
26,TV,A,A,A
7,Graphics Card,A,A,A


    Подводим итоги по полученной таблице
Товары A по всем метрикам:
Mouse, SSD, Smart Light, USB Drive, Smartphone, Laptop, TV, Graphics Card, Game Console, Tablet, External HDD,
E-book Reader, Drone, Charger, Headphones, RAM
Эти товары — золотой фонд. Их стоит: держать в фокусе маркетинга / не допускать дефицита /возможно, расширять ассортимент в этих категориях.


    Интересные случаи:
Speaker — B по количеству, но A по выручке и прибыли
→ Может продаваться реже, но по высокой цене и марже.
Router — A по продажам и прибыли, но B по выручке
→ Часто продаётся, но возможно, по невысокой цене.
Processor — A по продажам и выручке, но B по прибыли
→ Хороший оборот, но маржа низкая — стоит пересмотреть цену или закупку.
Microphone — A по продажам, но B по выручке и прибыли
→ Объем есть, но прибыль низкая — может стоит сделать апсейл.
    

    Класс C — слабые звенья:
Monitor, Keyboard, Motherboard, Fitness Tracker, Webcam
→ Продаются нечасто, приносят мало денег и прибыли.

    Рекомендации:
проанализировать: это запасной ассортимент или залежавшийся?
подумать о распродаже или выводе / попробовать связать с другими товарами (бандлы, апсейл).

    Вывод  
Стоит сфокусировать усилия на товарах из категории A/A/A.
Проверять несоответствия: если товар хорошо продаётся, но не приносит прибыль — пересмотреть ценообразование.
Товары из категории C — кандидаты на оптимизацию ассортимента или вывод из ассортимента.